In [1]:
import tensorflow as tf
import numpy as np
import time

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
from IcGanLayer import gan_loss, classify_loss, generator
from dataset_and_utils import prepare_datasets

In [11]:
n_classes = 444
batch_size = 4
n_iters = 169549*10//batch_size

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement=True

In [3]:
train_dataset, val_dataset, test_dataset = prepare_datasets()

Duration is 0.17427349090576172
Duration is 0.13858246803283691
Duration is 0.1507406234741211


In [17]:
tf.reset_default_graph()
with tf.device('/device:GPU:0'):
    x = tf.placeholder(tf.float32, shape=[None, 64, 64,  3], name='x')
    y = tf.placeholder(tf.float32, shape=[None, n_classes], name='y')
    
    loss_pred, loss_gen, gen = gan_loss(x, y, n_classes)
    loss_cls, Ey = classify_loss(x, y, n_classes)
    
#     saver = tf.train.Saver()
    evaluate = tf.equal(tf.round(Ey), y)
    accuracy = tf.reduce_mean(tf.cast(evaluate,dtype=tf.float32))
    
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        pred_optimizer = tf.train.AdamOptimizer(0.0002, beta1=0.5).minimize(loss_pred)
        gen_optimizer = tf.train.AdamOptimizer(0.0002, beta1=0.5).minimize(loss_gen)
        cls_optimizer = tf.train.AdamOptimizer(0.0002, beta1=0.5).minimize(loss_cls)
    
    

In [21]:
n_iters = 2
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for epoch in range(n_iters):
        cur_x, cur_y = train_dataset.next_batch(batch_size)
        print(cur_x.shape)
        print(cur_y.shape)
        pred_cost, gen_cost = sess.run([pred_optimizer, gen_optimizer], feed_dict={x:cur_x, y:cur_y})
        print('discriminator loss: {}, generator loss: {}'.format(pred_cost, gen_cost))
    cur_x, cur_y = train_dataset.next_batch(batch_size)
    test_image = sess.run([gen], feed_dict={x:test_dataset.next_batch(1), y:test_label.next_batch(1)})
#     show_image(test_image)
#     saver.save(sess, 'save/to/path')

(4, 64, 64, 3)
(4, 444)
discriminator loss: None, generator loss: None
(4, 64, 64, 3)
(4, 444)
discriminator loss: None, generator loss: None


NameError: name 'test_label' is not defined

In [15]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10606642384072603608
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10921135309
locality {
  bus_id: 2
}
incarnation: 13188180803565678751
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:85:00.0, compute capability: 6.1"
]


In [ ]:
with tf.Session(config=config) as sess:
    for epoch in range(n_iters):
        cls_cost = sess,run([cls_optimizer], feed_dict={x:dataset.nextbatch(), y:label.nextbatch()})
        print('classifer loss: %f, accuracy: %f' % (cls_cost, accuracy.eval()))
    saver.save(sess, 'save/to/path')
        